In [43]:
import random

def generate_data():
    n = 10

    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for i in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            # weight = random.uniform(0, 10)  # Adjust the range as needed
            weight = random.randint(0, 10)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [44]:
W = []
for i in range(1000):
    data = generate_data()
    W.append(data)


In [45]:
import numpy as np
array = np.array(data)

In [46]:
array.shape

(10, 10)

In [47]:
def randomize_set(W):
    A = []
    B = []

    for i in range(len(W)):
        r = random.randint(0,1)
        if r == 0:
            A.append(i)
        else:
            B.append(i)
    return A, B

In [48]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [49]:
A,B = randomize_set(W[0])
print("The (sub)optimal set A is {}".format(A))
print("The (sub)optimal set B is {}".format(B))
total_weight = count_weight(W[0], A, B)
print("The total weight of these sets is: {}".format(total_weight))

The (sub)optimal set A is [0, 1, 2, 3, 4, 5, 6, 7]
The (sub)optimal set B is [8, 9]
The total weight of these sets is: 73


In [50]:
def Greedy_Heuristic(W: list, A: list, B: list, total_weight: int):
    N = 500
    for i in range(N):
        r = random.randint(0,len(W)-1)
        temp_A = A.copy()
        temp_B = B.copy()
        # print("the old set A is: {}".format(A))
        # print("the old set B is: {}".format(B))
        if r in A:
            # print(str(r)+" is in A")
            temp_B.append(r)
            temp_A.remove(r)
        else:
            # print(str(r)+" is in B")
            temp_A.append(r)
            temp_B.remove(r)
        new_weight = count_weight(W, temp_A, temp_B)
        # print("the new weight is: {}".format(new_weight))
        # print("the old weight is: {}".format(total_weight))
        # print("the new set A is: {}".format(temp_A))
        # print("the new set B is: {}".format(temp_B))
        if new_weight >= total_weight:
            # print('success this new set is an improvement')
            A = temp_A
            B = temp_B
            total_weight = new_weight
        # print()
    A = sorted(A)
    B = sorted(B)
    # print("The optimal set A is {}".format(A))
    # print("The optimal set B is {}".format(B))
    total_weight = count_weight(W, A, B)
    # print("The total weight of these sets is: {}".format(total_weight))
    return A, B, total_weight

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def build_binary_model(xtrain, ytrain, xtest, ytest, nrlayers, nrnodes, rate):
    # This function builds the model with nrlayes layers and nrnodes nodes in each layer
    # nrlayers is a number and nrnodes is a vector specifying how many nodes each layer should contain
    model = keras.Sequential()
    numclasses = 1

    # add the layers
    for i in range(nrlayers + 1):
        if (i == 0):
            # input layer
            model.add(keras.Input(shape=input_shape))
        else:
            # hidden layers
            nrnode = nrnodes[i - 1]
            model.add(keras.layers.Dense(nrnode, activation="sigmoid"))

    model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

    # give summary of model
    model.summary()

    # train model
    batch_size = 32 # Based on SGD --> 1 gradient is calculated using 32 data points every time
    #so in each epoch, training_size/batch_size many gradient calculations and weight updates
    epochs = 10 # How many times to go through the data to complete the SGD

    opt = keras.optimizers.SGD(learning_rate=rate)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])

    # fit the model, note that keras takes care of the train set/validation set
    model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1)

    # get results
    score = model.evaluate(xtest, ytest, verbose=1)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    return model

In [51]:
set_A = []
set_B = []
sum_weight = []
for w in W:
    A,B = randomize_set(w)
    current_weight = count_weight(w, A, B)
    a,b,weight = Greedy_Heuristic(w, A, B, current_weight)
    set_A.append(a)
    set_B.append(b)
    sum_weight.append(weight)

In [64]:
g = [5,6,7,]
l = [5,7,3,2]

In [66]:
z = []
z.append(g)
z.append(l)

In [67]:
import numpy as np
test = np.array(z)


C:\Users\jelle\AppData\Local\Temp\ipykernel_11844\618559966.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test = np.array(z)


In [68]:
test

array([list([5, 6, 7]), list([5, 7, 3, 2])], dtype=object)